In [1]:
# import torch
import config
from model import Lightning_YOLO
from pytorch_lightning import Trainer, LightningDataModule

d:\Anaconda\envs\theschoolofai\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
import pytorch_lightning.utilities.model_summary

In [3]:
model = Lightning_YOLO(num_classes=config.NUM_CLASSES)

In [4]:
from pytorch_lightning.utilities import model_summary
# print(dir(model_summary))
summary = model_summary.ModelSummary(model)
print(summary)

  | Name    | Type       | Params
---------------------------------------
0 | layers  | ModuleList | 61.6 M
1 | loss_fn | YoloLoss   | 0     
---------------------------------------
61.6 M    Trainable params
0         Non-trainable params
61.6 M    Total params
246.506   Total estimated model params size (MB)


In [5]:
from utils import get_loaders
train_loader, test_loader, train_eval_loader = get_loaders(
    train_csv_path=config.DATASET + "/train.csv", test_csv_path=config.DATASET + "/test.csv"
)

In [6]:
class PascalDataModule(LightningDataModule):
    def __init__(self, data_dir: str = "./"):
        super().__init__()
        # self.data_dir = data_dir
        # self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    def prepare_data(self):
        pass
        # download
        # CIFAR10(self.data_dir, train=True, download=True)
        # CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage: str):
        pass
        # Assign train/val datasets for use in dataloaders
        # if stage == "fit":
        #     cifar_full = CIFAR10(self.data_dir, train=True, transform=self.transform)
        #     self.cifar_train, self.cifar_val = random_split(cifar_full, [55000, 5000])

        # # Assign test dataset for use in dataloader(s)
        # if stage == "test":
        #     self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.transform)

        # if stage == "predict":
        #     self.cifar_predict = CIFAR10(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return train_eval_loader

    def test_dataloader(self):
        return test_loader

    def predict_dataloader(self):
        return test_loader


In [7]:
len(train_loader)

518

In [8]:
dm = PascalDataModule()

In [9]:
# initialize the trainer
trainer = Trainer(
    max_epochs = 20,
    precision="16-mixed"
)
trainer.fit(model,datamodule=dm)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\Anaconda\envs\theschoolofai\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | layers  | ModuleList | 61.6 M
1 | loss_fn | YoloLoss   | 0     
---------------------------------------
61.6 M    T

Epoch 0:   0%|          | 1/518 [00:36<5:17:07, 36.80s/it, v_num=7]        

d:\Anaconda\envs\theschoolofai\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 19: 100%|██████████| 518/518 [05:38<00:00,  1.53it/s, v_num=7]{'train loss': 3.9802334308624268, 'Obj train accuracy': '84.313728%', 'No obj train accuracy is': '97.865936%', 'Class train accuracy is': '80.392159%'}


`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 518/518 [05:44<00:00,  1.50it/s, v_num=7]


In [11]:
# trainer.save_checkpoint("lightingYoloV1.ckpt")